# ИССЛЕДОВАНИЕ ДАННЫХ HR-АГЕНТСТВА

### HR-агентство изучает тренды на рынке труда в IT. Компания хочет провести исследование на основе данных о зарплатах в сфере Data Science за 2020–2022 годы и получить некоторые выводы.

## В процессе своего анализа вы должны:

1. Выяснить, какие факторы влияют на зарплату специалиста Data Scientist.
2. А также ответить на ключевые вопросы HR-агентства:
- Наблюдается ли ежегодный рост зарплат у специалистов Data Scientist?
- Как соотносятся зарплаты Data Scientist и Data Engineer в 2022 году?
- Как соотносятся зарплаты специалистов Data Scientist в компаниях различных размеров?
- Есть ли связь между наличием должностей Data Scientist и Data Engineer и размером компании?
- Если вы найдёте в данных интересные закономерности, также отметьте их в своём анализе.

Продемонстрируйте использование разных тестов для проверки статистической значимости сделанных выводов:

- тесты для количественного признака:
- для одной выборки;
- для двух выборок;
- для нескольких выборок;
- тест для категориальных признаков.

Результатом вашей работы должен стать ноутбук (IPYNB-файл) с кодом для исследования, а также с выводами и рассуждениями, полученными на основе разведывательного анализа.

#### Описание

- work_year - Год, в котором была выплачена зарплата.
- experience_level - Опыт работы на этой должности в течение года со следующими возможными значениями:
* *EN* — Entry-level/Junior; 
* *MI* — Mid-level/Intermediate;
* *SE* — Senior-level/Expert;
* *EX* —  Executive-level/Director.

- employment_type - Тип трудоустройства для этой роли:
* *PT* — неполный рабочий день;
* *FT* — полный рабочий день;
* *CT* — контракт;
* *FL* — фриланс

- job_title - Роль, в которой соискатель работал в течение года.
- salary - Общая выплаченная валовая сумма заработной платы.
- salary_currency - Валюта выплачиваемой заработной платы в виде кода валюты ISO 4217.
- salary_in_usd - Зарплата в долларах США (валютный курс, делённый на среднее значение курса доллара США за соответствующий год через fxdata.foorilla.com).
- employee_residence - Основная страна проживания сотрудника в течение рабочего года в виде кода страны ISO 3166.

- remote_ratio - Общий объём работы, выполняемой удалённо. Возможные значения:
* *0* — удалённой работы нет (менее 20 %);
* *50* — частично удалённая работа;
* *100* — полностью удалённая работа (более 80 %)

- company_location - Страна главного офиса работодателя или филиала по контракту в виде кода страны ISO 3166
- company_size - Среднее количество людей, работавших в компании в течение года:
* *S* — менее 50 сотрудников (небольшая компания);
* *M* — от 50 до 250 сотрудников (средняя компания);
* *L* — более 250 сотрудников (крупная компания).

## 1.Загрузка и обработка данных

### 1.1 Импорт необходимых библиотек и загрузка данных

In [1]:
# импорт необходимых библиотек
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels import stats as sms
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data=pd.read_csv('data/ds_salaries.csv')
data.head(10) # смотрим первые 10 строк нашего сета

,Unnamed: 0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
5,5,2020,EN,FT,Data Analyst,72000,USD,72000,US,100,US,L
6,6,2020,SE,FT,Lead Data Scientist,190000,USD,190000,US,100,US,S
7,7,2020,MI,FT,Data Scientist,11000000,HUF,35735,HU,50,HU,L
8,8,2020,MI,FT,Business Data Analyst,135000,USD,135000,US,100,US,L
9,9,2020,SE,FT,Lead Data Engineer,125000,USD,125000,NZ,50,NZ,S


In [3]:
# устанавливаем уровень альа значимости
alpha=0.05
print("Уровень значимости alpha = {:.2f}".format(alpha))

Уровень значимости alpha = 0.05


### 1.2 Проверка данных на пустые значения, дубликаты и корректность типов данных

In [4]:
data.info() # смотрим краткое инфо по сэту

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607 entries, 0 to 606
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          607 non-null    int64 
 1   work_year           607 non-null    int64 
 2   experience_level    607 non-null    object
 3   employment_type     607 non-null    object
 4   job_title           607 non-null    object
 5   salary              607 non-null    int64 
 6   salary_currency     607 non-null    object
 7   salary_in_usd       607 non-null    int64 
 8   employee_residence  607 non-null    object
 9   remote_ratio        607 non-null    int64 
 10  company_location    607 non-null    object
 11  company_size        607 non-null    object
dtypes: int64(5), object(7)
memory usage: 57.0+ KB


In [5]:
# проверяем данные на дубликаты
dubl=data[data.duplicated()].shape[0]
print(dubl)

0


Вывод: в датасете 607 записей. Пустые значения и дубликаты отсутствуют. Не совсем понял что задумывалось в данном пункте, на первый взгляд каждому из признаков соотвествует свой тип данных

### 1.3 Неинформационные столбцы

Вывод: судя по всему информация из признаков "salary" и "salary_currency" абсолютно неинформативна, так как есть признак "salary_in_usd", который сводит их к общему знаменателю.
Так же 1-ый столбец не несет никакой информации

### 1.4 Классификация на категориальные и числовые признаки

In [20]:
# Для классификации признаков решил посмотреть на кол-во уникальных значений в признаках
year=data['work_year'].unique().__len__()
print("Кол-во уникальных значений в признаке 'год выплаты з/п'=", year)
job=data['job_title'].unique().__len__()
print("Кол-во уникальных значений в признаке 'роль, в которой соискатель работал в течении года'=", job)
country=data['employee_residence'].unique().__len__()
print("Кол-во уникальных значений в признаке 'основная страна проживания сотрудника'=", country)
company_location=data['company_location'].unique().__len__()
print("Кол-во уникальных значений в признаке 'страна глаавного офиса работодателя'=", company_location)

Кол-во уникальных значений в признаке 'год выплаты з/п'= 3
Кол-во уникальных значений в признаке 'роль, в которой соискатель работал в течении года'= 50
Кол-во уникальных значений в признаке 'основная страна проживания сотрудника'= 57
Кол-во уникальных значений в признаке 'страна глаавного офиса работодателя'= 50


Вывод: на самом деле можно было не производить выше описанную процедуру, а обратить внимание на инфо о нашем сэте, которое говорит о следующем.

К числовым признакам относятся:
- *Salary*, но я не буду им пользоваться, потому что не информативен
- *Salary_currency*, так же не информативен
- *Salary_in_usd* вроде как основной признак, вокруг которого будет строится анализ

К категориальным признакам относятся:

Все оставшиеся признаки. Прнадлежность признаков *experience_level, employment_type, remote_ratio, company_size* в принципе проглядывалась в описании признаков к заданию. Признаки *work_year, job_title, employee_residence, company_location* показали свою принадлежность проверкой проделанной выше а так же отсутствием в значениях вообще каких либо цифр

### 1.5 Основные статистические характеристики для признаков